### Tuning Impact

In [1]:
import os; os.chdir('/Users/sqian/MKTSRV')
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import re
import string
import xlrd
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.SmartsAlertsDataModel import SmartsAlertsDataModel

from src.data_models.smartshelper import metrics, fixType, unSMART, unSMARTbe

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 1) Import Alerts generated from Calibration Q-1

In [2]:
alert_cleaned_prior = unSMART(pd.read_excel('~/Documents/datafldr/smarts0_2018NovDec.xlsx'))
# alert_cleaned_prior.dtypes

-> type fixed, kept 37572 rows, 48 columns


#### 2) Import Alerts generated from Calibration Q

In [3]:
alerts_cleaned = unSMART(pd.read_excel('~/Documents/datafldr/smarts1_2018Dec.xlsx'))
# alert_cleaned_prior['Datetime'] = pd.to_datetime(alert_cleaned_prior['Date'].astype(str) + ' ' + 
#                                                  alert_cleaned_prior['Time'].astype(str))
# alert_cleaned_prior.dtypes

-> type fixed, kept 152 rows, 13 columns


#### 2) Create and Apply Filters

In [4]:
month = 12
year = 2018

filter_month_prior = alert_cleaned_prior['Datetime'].dt.month == month
filter_year_prior = alert_cleaned_prior['Datetime'].dt.year == year
filter_date_prior = filter_month_prior & filter_year_prior

filter_month = alerts_cleaned['Datetime'].dt.month == month
filter_year = alerts_cleaned['Datetime'].dt.year == year
filter_date = filter_month & filter_year

In [5]:
alerts_prior = alert_cleaned_prior.loc[filter_date_prior]
print('-> kept', alerts_prior.shape[0], 'of', alert_cleaned_prior.shape[0], 'records')

alerts = alerts_cleaned.loc[filter_date]
print('-> filtered on month, kept', alerts.shape[0], 'records of', alerts_cleaned.shape[0], 'records')

-> kept 19702 of 37572 records
-> filtered on month, kept 152 records of 152 records


#### 3) Impact of Tuning

In [7]:
before = alerts_prior.groupby(['AlertCode','ShortText_mod']).count()['AlertID'].rename(columns={'AlertID':'BEFORE'})
after = alerts.groupby(['AlertCode','ShortText_mod']).count()['AlertID'].rename(columns={'AlertID':'AFTER'})

beforeVSafter = pd.concat([before,after],join='outer',axis=1).fillna(0).astype(int).reset_index().rename(columns={'ShortText_mod':'Alert',0:'BEFORE',1:'AFTER'})
beforeVSafter['Alert'] = beforeVSafter['Alert'].str.title()

beforeVSafter

,AlertCode,Alert,BEFORE,AFTER
0,1001,Unusual Price Movement Intra-Day,76,15
1,2011,Unusual Volume Intra-Day,1,15
2,4009,Painting The Tape,25,0
3,4012,Price Driver,30,13
4,4022,Bait And Switch,14893,36
5,4023,Layering Repeat,1,1
6,4032,Multi Order Spoofing,826,41
7,4041,Wash Sale A-B-A,29,2
8,4042,Wash Sale A-B-A (Money Pass),3818,26
9,4045,Collusion(Money Pass),3,3
